In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv
import os


In [ ]:
Dir="/content/drive/MyDrive/Colab Notebooks/AI_ML/Projects/images_dataset"
cropped=os.path.join(Dir,"cropped")
# os.mkdir(cropped)

In [ ]:
img=cv.imread('/content/drive/MyDrive/Colab Notebooks/AI_ML/Projects/images_dataset/lionel_messi/10_MESSI_JUGADORS-WEB.jpg')
cv.imwrite("/content/drive/MyDrive/Colab Notebooks/AI_ML/Projects/images_dataset/cropped/masii.png",img)

True

In [ ]:
for celeb in os.listdir(Dir)[-2:-1]:
  print (celeb)

virat_kohli


In [ ]:
haar_cascade=cv.CascadeClassifier("haar_face.xml")
eye_cascade=cv.CascadeClassifier("haarcascade_eye.xml")
celeb_dict={}
for celeb in os.listdir(Dir)[-2:-1]:
  celeb_path=os.path.join(Dir,celeb)
  cropped_img_list=[]
  i=1
  for photo in os.listdir(celeb_path):
    img_path=os.path.join(celeb_path,photo)
    img=cv.imread(img_path)
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    for(x,y,w,h) in haar_cascade.detectMultiScale(gray,1.3,5):
      roi_gray=gray[y:y+h, x:x+w]
      roi_color=img[y:y+h, x:x+w]
      eyes = eye_cascade.detectMultiScale(roi_gray)
      new_dir=os.path.join(cropped,celeb)
      if len(eyes)>=2:
        if(not os.path.exists(new_dir)):
          os.mkdir(new_dir)
        new_path=os.path.join(new_dir,celeb+str(i)+".png")
        if(not cv.imwrite(new_path,roi_color)):
          break
        cropped_img_list.append(new_path)
        i+=1
  celeb_dict[celeb]=cropped_img_list

In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.4 MB/s eta 0:00:00


In [ ]:
import  pywt
def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv.cvtColor( imArray,cv.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255;
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0;

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H


In [ ]:
X,y=[],[]
class_dict={}
count=1;
for celeb in os.listdir(cropped):
  class_dict[celeb]=count
  celeb_path=os.path.join(cropped,celeb)
  for path in os.listdir(celeb_path) :
    img_path=os.path.join(celeb_path,path)
    img=cv.imread(img_path)
    scalled_raw_img = cv.resize(img, (32, 32))
    img_har = w2d(img,'db1',5)
    scalled_img_har = cv.resize(img_har, (32, 32))
    combined_img=np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
    X.append(combined_img)
    y.append(count)
  count+=1



In [ ]:
X = np.array(X).reshape(len(X), 4096).astype(float)
X.shape

(166, 4096)

In [ ]:
len(y)

166

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}

In [ ]:
from sklearn.pipeline import make_pipeline
scores = []
best_estimators = {}
for m,mp in model_params.items():
  pipe = make_pipeline(StandardScaler(), mp['model'])
  clf=GridSearchCV(pipe,mp['params'],cv=5,return_train_score=False)
  clf.fit(X,y)
  scores.append({
      'model':m,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_
  })
  best_estimators[m]=clf.best_estimator_

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.873619,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,random_forest,0.711230,{'randomforestclassifier__n_estimators': 10}
2,logistic_regression,0.891800,{'logisticregression__C': 1}


In [ ]:
best_clf=best_estimators['logistic_regression']
best_clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(C=1, solver='liblinear'))])

In [ ]:
import joblib
joblib.dump(best_clf,'saved_model.pkl')

['saved_model.pkl']

In [ ]:
import json
with open("class_dictionary.json","w") as f:
    f.write(json.dumps(class_dict))